In [24]:
import mlflow

In [25]:
#conexion hacia el server de mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

In [26]:
import pandas as pd
data=pd.read_csv('../Data/train.csv')

In [27]:
# Creacion del cliente
client = mlflow.tracking.MlflowClient()

In [28]:
experiments=client.search_experiments()

In [29]:
experiments

[<Experiment: artifact_location='mlflow-artifacts:/874987130233476581', creation_time=1729458112266, experiment_id='874987130233476581', last_update_time=1729458112266, lifecycle_stage='active', name='experiment_final', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/728848010930649610', creation_time=1729454568995, experiment_id='728848010930649610', last_update_time=1729454568995, lifecycle_stage='active', name='rent4', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/400600718257267342', creation_time=1729126467671, experiment_id='400600718257267342', last_update_time=1729126467671, lifecycle_stage='active', name='rent3', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/705688815352764605', creation_time=1729039652228, experiment_id='705688815352764605', last_update_time=1729039652228, lifecycle_stage='active', name='rent2', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/459878205301148418', creation_time=1729007424862, experiment_i

In [30]:
experiment=list(filter(lambda k:k.name=='experiment_final',experiments))[0]

In [31]:
experiment

<Experiment: artifact_location='mlflow-artifacts:/874987130233476581', creation_time=1729458112266, experiment_id='874987130233476581', last_update_time=1729458112266, lifecycle_stage='active', name='experiment_final', tags={}>

In [32]:
experiment_id =experiment.experiment_id

In [33]:
experiment_id

'874987130233476581'

In [35]:
runs=client.search_runs(experiment_ids=experiment_id,
                        filter_string="",
                        run_view_type=mlflow.entities.ViewType.ACTIVE_ONLY,
                        max_results=3,
                        order_by=["metrics.TT DESC"])# ordenar por R2 desc porque va de 0 a 1

In [36]:
runs

[<Run: data=<RunData: metrics={'MAE': 0.1993,
  'MAPE': 0.4552,
  'MSE': 0.1034,
  'R2': 0.3783,
  'RMSE': 0.3215,
  'RMSLE': 0.2271,
  'TT': 38.36}, params={'bootstrap': 'True',
  'ccp_alpha': '0.0',
  'criterion': 'squared_error',
  'max_depth': 'None',
  'max_features': '1.0',
  'max_leaf_nodes': 'None',
  'max_samples': 'None',
  'min_impurity_decrease': '0.0',
  'min_samples_leaf': '1',
  'min_samples_split': '2',
  'min_weight_fraction_leaf': '0.0',
  'monotonic_cst': 'None',
  'n_estimators': '100',
  'n_jobs': '-1',
  'oob_score': 'False',
  'random_state': '123',
  'verbose': '0',
  'warm_start': 'False'}, tags={'Run ID': '4c6e264e1efb4b24852beefd0ca8722b',
  'Run Time': '351.07',
  'Source': 'create_model',
  'URI': 'cbd59555',
  'USI': '86c6',
  'mlflow.log-model.history': '[{"run_id": "4c6e264e1efb4b24852beefd0ca8722b", '
                              '"artifact_path": "model", "utc_time_created": '
                              '"2024-10-20 23:44:42.648678", "model_uuid": 

In [37]:
runs=runs[0]

In [38]:
runs

<Run: data=<RunData: metrics={'MAE': 0.1993,
 'MAPE': 0.4552,
 'MSE': 0.1034,
 'R2': 0.3783,
 'RMSE': 0.3215,
 'RMSLE': 0.2271,
 'TT': 38.36}, params={'bootstrap': 'True',
 'ccp_alpha': '0.0',
 'criterion': 'squared_error',
 'max_depth': 'None',
 'max_features': '1.0',
 'max_leaf_nodes': 'None',
 'max_samples': 'None',
 'min_impurity_decrease': '0.0',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'monotonic_cst': 'None',
 'n_estimators': '100',
 'n_jobs': '-1',
 'oob_score': 'False',
 'random_state': '123',
 'verbose': '0',
 'warm_start': 'False'}, tags={'Run ID': '4c6e264e1efb4b24852beefd0ca8722b',
 'Run Time': '351.07',
 'Source': 'create_model',
 'URI': 'cbd59555',
 'USI': '86c6',
 'mlflow.log-model.history': '[{"run_id": "4c6e264e1efb4b24852beefd0ca8722b", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2024-10-20 23:44:42.648678", "model_uuid": '
                             

In [39]:
run_id=runs.data.tags['Run ID']

In [40]:
#como tenemos el id  podemos utilizarlo para obtener los artefactos
run_id

'4c6e264e1efb4b24852beefd0ca8722b'

In [42]:
import mlflow
logged_model = f'runs:/{run_id}/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
exclude_variables=['id', 'CustomerId']
loaded_model.predict(data.drop(columns=exclude_variables+['Exited']))

array([0.  , 0.01, 0.08, ..., 0.12, 0.01, 0.12])

In [45]:
#utilizar en el entorno de produccion
#registrar un modelo el mejor pase por el proceso de validacion
# el modelo se registra en el servidor de mlflow
# se registra en funcion de run_id y poder colocar un nombre

mlflow.register_model(f"runs:/{run_id}/model", "experiment_final_model")

Registered model 'experiment_final_model' already exists. Creating a new version of this model...
2024/10/20 19:49:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: experiment_final_model, version 2
Created version '2' of model 'experiment_final_model'.


<ModelVersion: aliases=[], creation_timestamp=1729471767579, current_stage='None', description='', last_updated_timestamp=1729471767579, name='experiment_final_model', run_id='4c6e264e1efb4b24852beefd0ca8722b', run_link='', source='mlflow-artifacts:/874987130233476581/4c6e264e1efb4b24852beefd0ca8722b/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [47]:
client.transition_model_version_stage(name='experiment_final_model',version=1,stage='Production')

<ModelVersion: aliases=[], creation_timestamp=1729469289934, current_stage='Production', description='', last_updated_timestamp=1729471816934, name='experiment_final_model', run_id='4c6e264e1efb4b24852beefd0ca8722b', run_link='', source='mlflow-artifacts:/874987130233476581/4c6e264e1efb4b24852beefd0ca8722b/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [48]:
#la carga del modelo en produccion es desde el estado de produccion
import mlflow
logged_model = f'models:/experiment_final_model/Production'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
exclude_variables=['id', 'CustomerId']
loaded_model.predict(data.drop(columns=exclude_variables+['Exited']))

array([0.  , 0.01, 0.08, ..., 0.12, 0.01, 0.12])